# Задание

Использовать полученные знания в части обучения моделей для обучения и подбора параметров в задачах из предыдущих ДЗ на выбор (мфо, задача с тем возьмёт ли человек кредит или нет)

В частности использовать:
- GaussianNB
- LinearRegression + PolynomialFeatures.
- При обучении использовать make_pipeline
- GridSearchCV для поиска параметров

# Подготовка данных

In [169]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

In [170]:
df = pd.read_csv('data/MFOcredit.csv',
    encoding='cp1251',
    sep=';',
    index_col='id',
    parse_dates=['date_start', 'date_end'])
df.head(5)

,date_start,date_end,gender,age,auto,housing,marstatus,regclient,jobtype,region,credits,children,delinq60plus
id,,,,,,,,,,,,,
1,2013-01-03,2013-01-12,Мужской,44,Нет,Собственное,Гражданский брак/женат/замужем,Нет,Официальное,Новосибирская область,Нет,Да,Нет
2,2013-01-03,2013-01-17,Мужской,21,Пропуск поля,Живут с родителями,Холост,Нет,Официальное,Кемеровская область юг,Да,Нет,Нет
3,2013-01-03,2013-01-17,Мужской,25,Пропуск поля,Собственное,Холост,Да,Официальное,Кемеровская область север,Пропуск поля,Нет,Нет
4,2013-01-03,2013-01-17,Женский,47,Пропуск поля,Собственное,Гражданский брак/женат/замужем,Да,Официальное,Кемеровская область север,Нет,Нет,Нет
5,2013-01-03,2013-01-17,Мужской,22,Нет,Арендуемое,Гражданский брак/женат/замужем,Нет,Официальное,Кемеровская область север,Да,Да,Нет


In [171]:
y = 'delinq60plus'

X = df.columns.to_list()
X.remove(y)

X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], train_size=0.75, random_state=123)

In [172]:
columns = ['gender', 'auto', 'housing', 'marstatus', 'regclient', 'jobtype', 'region', 'credits', 'children']

def replace_empties(dataframe):
    for column in columns:
        new_value = dataframe[column].value_counts().index[0]
        dataframe.loc[dataframe[column] == "Пропуск поля", column] = new_value
    
    return dataframe

In [173]:
X_train = replace_empties(X_train)
X_test = replace_empties(X_test)

In [174]:
def cast_dates(dataframe):
    dataframe.date_start = dataframe.date_start.astype(np.int64)
    dataframe.date_end = dataframe.date_end.astype(np.int64)

In [175]:
cast_dates(X_train)
cast_dates(X_test)

In [176]:
def coding_features(dataframe):
    dataframe = pd.concat([dataframe,
           pd.get_dummies(dataframe['gender'], prefix='gender'),
           pd.get_dummies(dataframe['auto'], prefix='auto'),
           pd.get_dummies(dataframe['housing'], prefix='housing'),
           pd.get_dummies(dataframe['marstatus'], prefix='marstatus'),
           pd.get_dummies(dataframe['regclient'], prefix='regclient'),
           pd.get_dummies(dataframe['jobtype'], prefix='jobtype'),
           pd.get_dummies(dataframe['region'], prefix='region'),
           pd.get_dummies(dataframe['credits'], prefix='credits'),
           pd.get_dummies(dataframe['children'], prefix='children')],
           axis=1)
    
    return dataframe.drop(['gender', 'auto', 'housing', 'marstatus', 'regclient', 'jobtype', 'region', 'credits', 'children'], axis=1)

In [177]:
X_train = coding_features(X_train)
X_test = coding_features(X_test)

In [178]:
y_train = np.where(y_train == "Да", 1, 0)
y_test = np.where(y_test == "Да", 1, 0)

y_train

array([0, 1, 0, ..., 1, 0, 1])

# GaussianNB

In [179]:
gnb_params = {'var_smoothing': np.logspace(0,-9, num=100)}

gnb_grid = GridSearchCV(estimator=GaussianNB(),
                 param_grid=gnb_params, 
                 cv=2)

gnb_grid.fit(X_train, y_train)

gnb_grid.best_params_

{'var_smoothing': 0.03511191734215131}

In [180]:
gnb = gnb_grid.best_estimator_

In [181]:
y_pred = gnb.predict(X_test)
roc_auc_score(y_test, y_pred)

0.7104046440427967

# LinearRegression + PolynomialFeatures

In [183]:
pipe_params = [
    {
        'polynomialfeatures__degree': (1, 2, 3),
        'polynomialfeatures__include_bias': (True, False)
    },
    {
    }
]

pipe_grid = GridSearchCV(estimator=make_pipeline(PolynomialFeatures(), LinearRegression()),
                       param_grid=pipe_params,
                       n_jobs=-1,
                       cv=2)
pipe_grid.fit(X_train, y_train)

pipe_grid.best_params_

{'polynomialfeatures__degree': 2, 'polynomialfeatures__include_bias': False}

In [184]:
poly_model = pipe_grid.best_estimator_

In [185]:
y_pred = poly_model.predict(X_test)
roc_auc_score(y_test, y_pred)

0.8226530853417545